In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import csv
from sklearn.svm import SVC
from itertools import product
from sklearn.metrics import accuracy_score, confusion_matrix

with open('benchmarkdataset_train.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
bench_train = pd.DataFrame(data)

print(bench_train)

print()
# 計算每種胺基酸的出現次數
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
count_matrix_train = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_train[i, j] = sequence.count(amino_acid)
    
    percentage_matrix_train.append(count_matrix_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_train = np.array(percentage_matrix_train)
print('percentage_train in 1-D:')
print(percentage_matrix_train)

print()
# 顯示比例矩陣
df_percentage_train = pd.DataFrame(percentage_matrix_train, columns=list(amino_acids))
print(df_percentage_train)

# 合併特徵資料和目標資料
df_combined_train = pd.concat([df_percentage_train, bench_train['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_train['Target'] = df_combined_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_train, y_train = df_combined_train.drop(['Header', 'Target'], axis=1), df_combined_train['Target']

#print(X_train)
#print(y_train)

#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_train = np.zeros((len(sequences), len(DPC)))
percentage_DPC_train = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_train[i, j] = sequence.count(aa_pair)
        
    percentage_DPC_train.append(count_DPC_train[i]/len(sequence))

# 計算每對胺基酸在序列中的比例
percentage_DPC_train = np.array(percentage_DPC_train)
print('percentage_DPC_train in 1-D:')
print(percentage_DPC_train)

# 顯示比例矩陣
df_percentage_DPC_train = pd.DataFrame(percentage_DPC_train, columns=DPC)
print(df_percentage_DPC_train)
print()

# 合併特徵資料和目標資料
df_combined_DPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, bench_train['Header']], axis=1)
#df_combined_DPC_train = pd.concat([df_percentage_DPC_train, bench_train['Header']], axis=1)
print(df_combined_DPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_train['Target'] = df_combined_DPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_train, y_DPC_train = df_combined_DPC_train.drop(['Header', 'Target'], axis=1), df_combined_DPC_train['Target']
#print(X_DPC_train)

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_train = np.zeros((len(sequences), len(TPC)))
percentage_TPC_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_train[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_train.append(count_TPC_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_TPC_train = np.array(percentage_TPC_train)
print('percentage_TPC_train in 1-D:')
print(percentage_TPC_train)

# 顯示比例矩陣
df_percentage_TPC_train = pd.DataFrame(percentage_TPC_train, columns=TPC)
print(df_percentage_TPC_train)

# 合併特徵資料和目標資料
df_combined_TPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, df_percentage_TPC_train, bench_train['Header']], axis=1)
print(df_combined_TPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_train['Target'] = df_combined_TPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_train, y_TPC_train = df_combined_TPC_train.drop(['Header', 'Target'], axis=1), df_combined_TPC_train['Target']

ModuleNotFoundError: No module named 'pandas'

In [ ]:
with open('benchmarkdataset_test.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
bench_test = pd.DataFrame(data)

#print(bench_test)

print()
# 計算每種胺基酸的出現次數
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
count_matrix_test = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_test[i, j] = sequence.count(amino_acid)
    percentage_matrix_test.append(count_matrix_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_test = np.array(percentage_matrix_test)
print('percentage_test in 1-D:')
print(percentage_matrix_test)

print()
# 顯示比例矩陣
df_percentage_test = pd.DataFrame(percentage_matrix_test, columns=list(amino_acids))
#print(df_percentage_test)

# 合併特徵資料和目標資料
df_combined_test = pd.concat([df_percentage_test, bench_test['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_test['Target'] = df_combined_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_test, y_test = df_combined_test.drop(['Header', 'Target'], axis=1), df_combined_test['Target']

#print(X_test)
#print(y_test)
#print(y_test.value_counts())

#--------------------------------------------------------------
# 計算每對胺基酸的出現次數
count_DPC_test = np.zeros((len(sequences), len(DPC)))
percentage_DPC_test = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_test[i, j] = sequence.count(aa_pair)
        
    percentage_DPC_test.append(count_DPC_test[i]/len(sequence))

# 計算每對胺基酸在序列中的比例
percentage_DPC_test = np.array(percentage_DPC_test)
print('percentage_DPC_train in 1-D:')
print(percentage_DPC_test)

# 顯示比例矩陣
df_percentage_DPC_test = pd.DataFrame(percentage_DPC_test, columns=DPC)
print(df_percentage_DPC_test)

# 合併特徵資料和目標資料
df_combined_DPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, bench_test['Header']], axis=1)
#df_combined_DPC_test = pd.concat([df_percentage_DPC_test, bench_test['Header']], axis=1)
print(df_combined_DPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_test['Target'] = df_combined_DPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_test, y_DPC_test = df_combined_DPC_test.drop(['Header', 'Target'], axis=1), df_combined_DPC_test['Target']

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_test = np.zeros((len(sequences), len(TPC)))
percentage_TPC_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_test[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_test.append(count_TPC_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_TPC_test = np.array(percentage_TPC_test)
print('percentage_TPC_test in 1-D:')
print(percentage_TPC_test)

# 顯示比例矩陣
df_percentage_TPC_test = pd.DataFrame(percentage_TPC_test, columns=TPC)
print(df_percentage_TPC_test)

# 合併特徵資料和目標資料
df_combined_TPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, df_percentage_TPC_test, bench_test['Header']], axis=1)
print(df_combined_TPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_test['Target'] = df_combined_TPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_test, y_TPC_test = df_combined_TPC_test.drop(['Header', 'Target'], axis=1), df_combined_TPC_test['Target']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')

print('-------------------------------------------------')

rf_classifier.fit(X_DPC_train, y_DPC_train)
y_DPC_pred = rf_classifier.predict(X_DPC_test)

accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')

print('-------------------------------------------------')

rf_classifier.fit(X_TPC_train, y_TPC_train)
y_TPC_pred = rf_classifier.predict(X_TPC_test)

accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')

In [ ]:
with open('NT15dataset_train.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
nt15_train = pd.DataFrame(data)

print(nt15_train)

print()
# 計算每種胺基酸的出現次數
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
count_matrix_train = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_train[i, j] = sequence.count(amino_acid)
    
    percentage_matrix_train.append(count_matrix_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_train = np.array(percentage_matrix_train)
print('percentage_train in 1-D:')
print(percentage_matrix_train)

print()
# 顯示比例矩陣
df_percentage_train = pd.DataFrame(percentage_matrix_train, columns=list(amino_acids))
print(df_percentage_train)

# 合併特徵資料和目標資料
df_combined_train = pd.concat([df_percentage_train, nt15_train['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_train['Target'] = df_combined_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_train, y_train = df_combined_train.drop(['Header', 'Target'], axis=1), df_combined_train['Target']

#print(X_train)
#print(y_train)

#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_train = np.zeros((len(sequences), len(DPC)))
percentage_DPC_train = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_train[i, j] = sequence.count(aa_pair)
        
    percentage_DPC_train.append(count_DPC_train[i]/15.)

# 計算每對胺基酸在序列中的比例
percentage_DPC_train = np.array(percentage_DPC_train)
print('percentage_DPC_train in 1-D:')
print(percentage_DPC_train)

# 顯示比例矩陣
df_percentage_DPC_train = pd.DataFrame(percentage_DPC_train, columns=DPC)
print(df_percentage_DPC_train)

# 合併特徵資料和目標資料
df_combined_DPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, nt15_train['Header']], axis=1)
#df_combined_DPC_train = pd.concat([df_percentage_DPC_train, nt15_train['Header']], axis=1)
print(df_combined_DPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_train['Target'] = df_combined_DPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_train, y_DPC_train = df_combined_DPC_train.drop(['Header', 'Target'], axis=1), df_combined_DPC_train['Target']
#print(X_DPC_train)

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_train = np.zeros((len(sequences), len(TPC)))
percentage_TPC_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_train[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_train.append(count_TPC_train[i]/15.)

# 計算每種胺基酸在序列中的比例
percentage_TPC_train = np.array(percentage_TPC_train)
print('percentage_TPC_train in 1-D:')
print(percentage_TPC_train)

# 顯示比例矩陣
df_percentage_TPC_train = pd.DataFrame(percentage_TPC_train, columns=TPC)
print(df_percentage_TPC_train)

# 合併特徵資料和目標資料
df_combined_TPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, df_percentage_TPC_train, nt15_train['Header']], axis=1)
print(df_combined_TPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_train['Target'] = df_combined_TPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_train, y_TPC_train = df_combined_TPC_train.drop(['Header', 'Target'], axis=1), df_combined_TPC_train['Target']

In [ ]:
with open('NT15dataset_test.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
nt15_test = pd.DataFrame(data)

#print(nt15_test)

print()
# 計算每種胺基酸的出現次數
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
count_matrix_test = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_test[i, j] = sequence.count(amino_acid)
    
    percentage_matrix_test.append(count_matrix_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_test = np.array(percentage_matrix_test)
print('percentage_test in 1-D:')
print(percentage_matrix_test)

print()
# 顯示比例矩陣
df_percentage_test = pd.DataFrame(percentage_matrix_test, columns=list(amino_acids))
print(df_percentage_test)

# 合併特徵資料和目標資料
df_combined_test = pd.concat([df_percentage_test, nt15_test['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_test['Target'] = df_combined_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_test, y_test = df_combined_test.drop(['Header', 'Target'], axis=1), df_combined_test['Target']

#print(X_test)
#print(y_test)

#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_test = np.zeros((len(sequences), len(DPC)))
percentage_DPC_test = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_test[i, j] = sequence.count(aa_pair)
        
    percentage_DPC_test.append(count_DPC_test[i]/15.)

# 計算每對胺基酸在序列中的比例
percentage_DPC_test = np.array(percentage_DPC_test)
print('percentage_DPC_test in 1-D:')
print(percentage_DPC_test)

# 顯示比例矩陣
df_percentage_DPC_test = pd.DataFrame(percentage_DPC_test, columns=DPC)
print(df_percentage_DPC_test)

# 合併特徵資料和目標資料
df_combined_DPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, nt15_test['Header']], axis=1)
#df_combined_DPC_test = pd.concat([df_percentage_DPC_test, nt15_test['Header']], axis=1)
print(df_combined_DPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_test['Target'] = df_combined_DPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_test, y_DPC_test = df_combined_DPC_test.drop(['Header', 'Target'], axis=1), df_combined_DPC_test['Target']
#print(X_DPC_train)

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_test = np.zeros((len(sequences), len(TPC)))
percentage_TPC_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_test[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_test.append(count_TPC_test[i]/15.)

# 計算每種胺基酸在序列中的比例
percentage_TPC_test = np.array(percentage_TPC_test)
print('percentage_TPC_test in 1-D:')
print(percentage_TPC_test)

# 顯示比例矩陣
df_percentage_TPC_test = pd.DataFrame(percentage_TPC_test, columns=TPC)
print(df_percentage_TPC_test)

# 合併特徵資料和目標資料
df_combined_TPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, df_percentage_TPC_test, nt15_test['Header']], axis=1)
print(df_combined_TPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_test['Target'] = df_combined_TPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_test, y_TPC_test = df_combined_TPC_test.drop(['Header', 'Target'], axis=1), df_combined_TPC_test['Target']

from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')

print('-------------------------------------------------')

rf_classifier.fit(X_DPC_train, y_DPC_train)
y_DPC_pred = rf_classifier.predict(X_DPC_test)

accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')

print('-------------------------------------------------')

rf_classifier.fit(X_TPC_train, y_TPC_train)
y_TPC_pred = rf_classifier.predict(X_TPC_test)

accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')